In [1]:
path_to_src = '/content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src'
dataset_name = "QM9"
task_lst = ["alpha","mu","homo","lumo","gap","r2","zpve","u0","u298","h298","g298","cv"]
dataset_metrics = ['r2','mae']
"""dataset_name = "PCBA"
task_lst = ["PCBA-1030","PCBA-1458","PCBA-1460","PCBA-2546","PCBA-2551","PCBA-485297","PCBA-485313","PCBA-485364","PCBA-504332","PCBA-504333","PCBA-504339","PCBA-504444","PCBA-504467","PCBA-588342","PCBA-624296","PCBA-624297","PCBA-624417","PCBA-651965","PCBA-652104","PCBA-686970","PCBA-686978","PCBA-686979","PCBA-720504"]
dataset_metrics = ['roc_auc','auprc']"""

#If using Google drive
from google.colab import drive
drive.mount('/content/drive')
# get the current working directory
import sys
sys.path.insert(0,path_to_src)

Mounted at /content/drive


In [2]:
#!pip3 uninstall dgl -y
!pip3 install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip3 install dgllife
!pip3 install rdkit

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 39.7 MB/s eta 0:00:00


## Before moving on, make sure the "Calculate Similarity" section of motse.ipynb has been run for the dataset of interest

In [3]:
import torch
import pandas as pd
from utils import load_model, makedir, set_random_seed
from utils.data import load_data
from multi_trainer import Multi_Trainer
from trainer import Trainer

%load_ext autoreload
%autoreload 2
set_random_seed(18) #It's been run on 22 for both

task_lst = task_lst[:10] #Only testing transfer to first 10 tasks

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
#Pick best 3 source tasks for each target task based on MSE similarity

data_path = path_to_src + f'/../datasets/{dataset_name}/1000/'
model_type = 'GCN'
first_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-1/" #To save model trained in tandem w/ first source task
second_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-2/" #To save model trained in tandem w/ second source task
third_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-3/" #To save model trained in tandem w/ third source task
first_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-1/" #To save finetuned model trained in tandem w/ first source task
second_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-2/" #To save finetuned model trained in tandem w/ second source task
third_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-3/" #To save finetuned model trained in tandem w/ third source task
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = dict()

source_target_pairs = []
similarity_results_path = path_to_src + f"/../results/{dataset_name}/GCN/10000/zinc500/"
for target_task in task_lst:
    similarity_df = pd.read_csv(similarity_results_path + target_task + '.csv')
    similarity_df = similarity_df.sort_values(by='motse', ascending=False)
    source_task1 = similarity_df.iloc[[1]]['source_task'].to_list()[0]
    source_task2 = similarity_df.iloc[[2]]['source_task'].to_list()[0]
    source_task3 = similarity_df.iloc[[3]]['source_task'].to_list()[0]
    source_target_pairs.append((source_task1, source_task2, source_task3, target_task))

In [5]:
def multi_train(source_target_pairs, source_number, source_model_path, save_model_path):
  makedir(save_model_path)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  results_df = pd.DataFrame(columns = ['task'] + dataset_metrics)
  for source_task1, source_task2, source_task3, target_task in source_target_pairs:
    if source_number == 1:
      source_task = source_task1
      prev_source_task = ''
    elif source_number == 2:
      source_task = source_task2
      prev_source_task = source_task1
    else:
      source_task = source_task3
      prev_source_task = source_task2
    #Train w/ first source task
    print('multi ' + target_task + ' ' + source_task)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[target_task],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task],
        device = device
    )
    if source_model_path is None:
      model = load_model(device=device, is_multi = True)
    else:
      model = load_model(device=device, is_multi = True, source_model_path = source_model_path+f"{target_task}.pth")
    trainer = Multi_Trainer(device=device,task1=target_task,task2=source_task,
                      data_args1=data_args1,data_args2=data_args2,model_path=save_model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)
    result = {'task': target_task}
    for metric in data_args1['metrics']:
        if metric in dataset_metrics:
          result[metric] = task_results1_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = save_model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
  print(f"Results have been saved to {result_path+'results.csv'}")

def multi_ft(source_target_pairs, source_number, source_model_path, save_model_path):
  makedir(save_model_path)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  if source_number == 1:
    results_df = pd.DataFrame(columns = ['task', 'source1'] + dataset_metrics)
  elif source_number == 2:
    results_df = pd.DataFrame(columns = ['task', 'source1', 'source2'] + dataset_metrics)
  else:
    results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'source3'] + dataset_metrics)
  for source_task1, source_task2, source_task3, target_task in source_target_pairs:
      if source_number == 1:
        source_task = source_task1
        result = {'task': target_task, 'source1': source_task1}
      elif source_number == 2:
        source_task = source_task2
        result = {'task': target_task, 'source1': source_task1, 'source2': source_task2}
      else:
        source_task = source_task3
        result = {'task': target_task, 'source1': source_task1, 'source2': source_task2, 'source3': source_task3}
      print(f"ft on {target_task}")

      train_loader, val_loader, test_loader, data_args = load_data(
          dataset=dataset_name,data_path=data_path,tasks=[target_task],
          device = device
      )
      model = load_model(
          device=device,
          source_model_path=source_model_path+f"{target_task}_{source_task}.pth",
          is_multi = True
      )
      trainer = Trainer(
          device=device,tasks=[target_task],
          data_args=data_args,model_path=save_model_path)
      _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
      for metric in data_args['metrics']:
          if metric in dataset_metrics:
            result[metric] = task_results_dict[metric][target_task]
      results_df = results_df.append(result, ignore_index=True)
      result_path = save_model_path.replace('saved_models','results')
      makedir(result_path)
      results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
  print(f"Results have been saved to {result_path+'results.csv'}")

In [6]:
#Pre-train GNN w/ target task and first source task
#Use target data to quickly fine-tune model pre-trained on one source task + target (for comparison purposes w/ using 2 and 3 tasks)

multi_train(source_target_pairs, 1, None, first_train_model_path)
multi_ft(source_target_pairs, 1, first_train_model_path, first_ft_model_path)

multi alpha zpve
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
initializing trainer...
[0] training loss:1.7217995858192443
alpha
val r2:0.2258912038567059
val mae:5.698385238647461
zpve
val r2:0.2897929658737506
val mae:0.023690586909651756
[20] training loss:0.8477389001846314
alpha
val r2:0.6149076086614864
val mae:3.997546911239624
zpve
val r2:0.5415742301953669
val mae:0.01928115449845791
[40] training loss:0.7903722095489502
alpha
val r2:0.6625091789955546
val mae:3.7170889377593994
zpve
val r2:0.5789963416063304
val mae:0.018311303108930588
[60] training loss:0.7543962329626084
alpha
val r2:0.6501411834744748
val mae:3.812943935394287
zpve
val r2:0.5797042119953579
val mae:0.018170980736613274
[80] training loss:0.7353733545541763
alpha
val r2:0.6935063216174924
val mae:3.6159451007843018
zpve
val r2:0.6137488198359389
val mae:0.017609477043151855
[100] training loss:0.7005735397338867
alpha
val r2:0.7246270110687963
val mae:3.4671993255615234
zpve
va

<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
initializing trainer...
[0] training loss:1.948799329996109
mu
val r2:0.11638393602260044
val mae:1.1687332391738892
r2
val r2:0.01745719137834889
val mae:227.19393920898438
[20] training loss:1.5815244829654693
mu
val r2:0.31035431327557217
val mae:0.9935188293457031
r2
val r2:0.08863899872230119
val mae:206.1924591064453
[40] training loss:1.5033174359798431
mu
val r2:0.3454062931027384
val mae:0.9448121786117554
r2
val r2:0.19118049915801139
val mae:216.27188110351562
[60] training loss:1.4349596071243287
mu
val r2:0.31764580259866326
val mae:0.9874061346054077
r2
val r2:0.1572431708618438
val mae:193.7707061767578
[80] training loss:1.3850954043865205
mu
val r2:0.34771554171343544
val mae:0.9446072578430176
r2
val r2:0.25397903345165496
val mae:193.84634399414062
[100] training loss:1.3417552566528321
mu
val r2:0.3417169220568216
val mae:0.9434932470321655
r2
val r2:0.26426632599451183
val mae:190.29937744140625
[120] training loss:

<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
initializing trainer...
[0] training loss:1.9004174327850343
homo
val r2:0.027355633619430386
val mae:0.016385076567530632
lumo
val r2:0.23990964736137232
val mae:0.030658969655632973
[20] training loss:1.6270227718353272
homo
val r2:0.11422554315102329
val mae:0.01527650747448206
lumo
val r2:0.28153108382109504
val mae:0.03168370947241783
[40] training loss:1.5599847102165223
homo
val r2:0.2043334451621659
val mae:0.014769915491342545
lumo
val r2:0.3253798858095762
val mae:0.030462488532066345
[60] training loss:1.4781321799755096
homo
val r2:0.2585847613458474
val mae:0.014407304115593433
lumo
val r2:0.3375035674327692
val mae:0.030219685286283493
[80] training loss:1.451061805486679
homo
val r2:0.2736502818710298
val mae:0.014122094959020615
lumo
val r2:0.382373855382922
val mae:0.029477350413799286
test r2:0.22309312184631824
test mae:0.014348438940942287
test r2:0.22510256990914845
test mae:0.032977595925331116
multi lu

...

<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-1/u298_g298.pth
[0] training loss:0.015782279670238496
val r2:0.9854431484635907
val mae:4.378787040710449
[20] training loss:0.012533618900924921
val r2:0.9874263760333093
val mae:4.0190205574035645
test r2:0.973618495856875
test mae:3.837799549102783
ft on h298
800 loaded!
100 loaded!


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-1/h298_u0.pth
[0] training loss:0.017756736483424903
val r2:0.9888469781678032
val mae:3.799160957336426
[20] training loss:0.012289019054733216
val r2:0.9848260258365451
val mae:4.5818891525268555
[40] training loss:0.010905492696911096
val r2:0.9869748745976401
val mae:3.9940199851989746
test r2:0.9771848171682165
test mae:3.9087157249450684
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/QM9/GCN/multi1000-1/results.csv


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [7]:
#Pre-train GNN w/ target task and second source task, using pre-trained weights from target task + first source task
#Use target data to fine-tune model pre-trained on 2 source tasks + target

multi_train(source_target_pairs, 2, first_ft_model_path, second_train_model_path)
multi_ft(source_target_pairs, 2, second_train_model_path, second_ft_model_path)

multi alpha r2
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-1/alpha.pth
initializing trainer...
[0] training loss:1.0134540528059006
alpha
val r2:0.8096732500849051
val mae:2.798077344894409
r2
val r2:0.2300843542113884
val mae:199.36746215820312
[20] training loss:0.807865903377533
alpha
val r2:0.816391781129941
val mae:2.7403481006622314
r2
val r2:0.3273193873766227
val mae:172.91983032226562
[40] training loss:0.7864297819137573
alpha
val r2:0.7683477711306248
val mae:3.1127796173095703
r2
val r2:0.34445472057346693
val mae:171.27183532714844
[60] training loss:0.7257879298925399
alpha
val r2:0.786899850921014
val mae:2.979985237121582
r2
val r2:0.4084916900178599
val mae:168.9673309326172
[80] training loss:0.7015468651056289
alpha
val r2:0.8183487327799427
val mae:2.7048709392547607
r2
val r2:0.4274025851176041
val mae:161.0318603515625

<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-1/mu.pth
initializing trainer...
[0] training loss:1.2352983164787292
mu
val r2:0.2931498060876132
val mae:0.9742252826690674
cv
val r2:0.42392702432096996
val mae:2.6252477169036865
[20] training loss:1.0633302187919618
mu
val r2:0.290894357297655
val mae:0.9864967465400696
cv
val r2:0.5056874313438083
val mae:2.50278639793396
[40] training loss:0.9922354102134705
mu
val r2:0.2721670477980014
val mae:0.9855740070343018
cv
val r2:0.4958768822561602
val mae:2.53071928024292
test r2:0.36196714546903297
test mae:0.8454223871231079
test r2:0.46793686710223004
test mae:2.316737174987793
multi homo mu
800 loaded!
100 loaded!


<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-1/homo.pth
initializing trainer...
[0] training loss:1.8124617648124695
homo
val r2:0.2808417492634434
val mae:0.014346824958920479
mu
val r2:0.18119121887329415
val mae:1.0880742073059082
[20] training loss:1.4540750932693483
homo
val r2:0.28600975416603946
val mae:0.014272660948336124
mu
val r2:0.2778502282096924
val mae:1.0209711790084839
[40] training loss:1.395247507095337
homo
val r2:0.24424435470567507
val mae:0.014533000066876411
mu
val r2:0.32106842788800205
val mae:0.9885496497154236
[60] training loss:1.3432124555110931
homo
val r2:0.21286479657630974
val mae:0.015339594334363937
mu
val r2:0.3023496070731294
val mae:0.981336236000061
[80] training loss:1.288507306575775
homo
val r2:0.24786867993656247
val mae:0.014684661291539669
mu
val r2:0.29760669830433173
val mae:0.9916757345199585
test r2:0.15652227800204

...

<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-2/u298_h298.pth
[0] training loss:0.02066131350584328
val r2:0.9845891468546333
val mae:4.381060600280762
[20] training loss:0.009800036726519466
val r2:0.9883789347370263
val mae:3.839329242706299
test r2:0.9753601250354295
test mae:3.8876709938049316
ft on h298
800 loaded!
100 loaded!


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-2/h298_g298.pth
[0] training loss:0.01882432207465172
val r2:0.9885095865752997
val mae:3.8057820796966553
[20] training loss:0.011506675602868199
val r2:0.9876243746722958
val mae:3.792750358581543
test r2:0.9799077884084205
test mae:3.669058084487915
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/QM9/GCN/multi1000-2/results.csv


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [8]:
#Pre-train GNN w/ target task and third source task, using pre-trained weights from target task + first two source tasks
#Use target data to fine-tune model pre-trained on 2 source tasks + target

multi_train(source_target_pairs, 3, second_ft_model_path, third_train_model_path)
multi_ft(source_target_pairs, 3, third_train_model_path, third_ft_model_path)

multi alpha cv
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-2/alpha.pth
initializing trainer...
[0] training loss:0.6504141628742218
alpha
val r2:0.8012020412538485
val mae:2.8630383014678955
cv
val r2:0.48995911801378933
val mae:2.505586624145508
[20] training loss:0.4756989443302155
alpha
val r2:0.8097016840985274
val mae:2.752713918685913
cv
val r2:0.5588308911420949
val mae:2.3228445053100586
[40] training loss:0.45575679302215577
alpha
val r2:0.8176142776153987
val mae:2.6807854175567627
cv
val r2:0.6025006385604235
val mae:2.2304134368896484
[60] training loss:0.4207127672433853
alpha
val r2:0.8150484833510382
val mae:2.7158803939819336
cv
val r2:0.5055529827789079
val mae:2.427227735519409
test r2:0.7860524933061631
test mae:2.8776302337646484
test r2:0.5887743624380319
test mae:2.0960044860839844


<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


multi mu homo
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-2/mu.pth
initializing trainer...
[0] training loss:1.7563804006576538
mu
val r2:0.3098679138413434
val mae:0.9460744261741638
homo
val r2:0.009228182784996042
val mae:0.01700269989669323
[20] training loss:1.298904811143875
mu
val r2:0.2683001481379844
val mae:0.9956852197647095
homo
val r2:0.2371586186757928
val mae:0.014551537111401558
[40] training loss:1.2310735833644868
mu
val r2:0.22184034652437545
val mae:1.0146090984344482
homo
val r2:0.24865904821750373
val mae:0.014464703388512135
[60] training loss:1.1688390028476716
mu
val r2:0.19898083189273852
val mae:1.0274559259414673
homo
val r2:0.21926824683579138
val mae:0.0152383241802454
[80] training loss:1.1084440916776657
mu
val r2:0.18995974557035844
val mae:1.012768268585205
homo
val r2:0.16749465396298346
val mae:0.01512959

<ipython-input-5-30e9c3db7a27>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000-2/homo.pth
initializing trainer...
[0] training loss:1.6615821516513825
homo
val r2:0.22427037836509833
val mae:0.014445223845541477
cv
val r2:0.11164492797540426
val mae:3.459784746170044
[20] training loss:1.3427681148052215
homo
val r2:0.22658447769426615
val mae:0.014594261534512043
cv
val r2:0.31240360435750647
val mae:2.9645895957946777
[40] training loss:1.2278921246528625
homo
val r2:0.24883017214247527
val mae:0.014723364263772964
cv
val r2:0.34978322464165146
val mae:2.9454801082611084
[60] training loss:1.1507403206825257
homo
val r2:0.26888794671259897
val mae:0.014450173825025558
cv
val r2:0.36272227549549496
val mae:2.8292276859283447
[80] training loss:1.1154843252897262
homo
val r2:0.26933801240167277
val mae:0.014392583630979061
cv
val r2:0.3596313052252439
val mae:2.8764357566833496
test r2:0.2263668463

...

<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-3/u298_u0.pth
[0] training loss:0.014172323448583483
val r2:0.9893977852404482
val mae:3.635509729385376
[20] training loss:0.011114656482823193
val r2:0.9875354113849031
val mae:3.707890510559082
test r2:0.9770790046743558
test mae:3.6597611904144287
ft on h298
800 loaded!
100 loaded!


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/raw_multi1000-3/h298_u298.pth
[0] training loss:0.015442090276628732
val r2:0.9881013141460494
val mae:4.186408519744873
[20] training loss:0.01107816061936319
val r2:0.9881478571964453
val mae:3.741558313369751
[40] training loss:0.00918258585035801
val r2:0.9886341760745432
val mae:3.563666343688965
test r2:0.9809242511851695
test mae:3.734110116958618
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/QM9/GCN/multi1000-3/results.csv


<ipython-input-5-30e9c3db7a27>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [10]:
#Coallate results

import pandas as pd
from utils import makedir

metric = dataset_metrics[0]
results_path = path_to_src + f"/../results/{dataset_name}/GCN/"
makedir(results_path + 'multi1000')

results_df1 = pd.read_csv(results_path + 'multi1000-1/results3.csv')
results_df2 = pd.read_csv(results_path + 'multi1000-2/results3.csv')
results_df3 = pd.read_csv(results_path + 'multi1000-3/results3.csv')
results_df = pd.concat([results_df1, results_df2, results_df3])
results_df = results_df[['task', metric]]

idx = results_df.groupby(['task'])[metric].transform(max) == results_df[metric]
results_df = results_df[idx]
results_df = results_df.drop_duplicates(['task']) #If two source tasks have the same score, just choose one randomly
results_df.to_csv(results_path + 'multi1000/results3.csv', index = False)

# OLD

In [ ]:
#Pre-train GNN w/ target task and first source task

model_type = 'GCN'
model_path = first_train_model_path
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[], 'helper_task':[]}

for source_task1, _, _, target_task in source_target_pairs:
    #Train w/ first source task
    print('multi ' + target_task + ' ' + source_task1)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[target_task],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task1],
        device = device
    )
    model = load_model(device=device, is_multi = True)
    trainer = Multi_Trainer(device=device,task1=target_task,task2=source_task1,
                      data_args1=data_args1,data_args2=data_args2,model_path=model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)

#Use target data to quickly fine-tune model pre-trained on one source task + target (for comparison purposes w/ using 2 and 3 tasks)

results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'r2', 'mae'])
for source_task1, source_task2, target_task in source_target_pairs:
    print(f"finetune on {target_task}")

    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,data_path=data_path,tasks=[target_task],
        device = device
    )
    model = load_model(
        device=device,
        source_model_path=first_train_model_path+f"{target_task}_{source_task1}.pth",
        is_multi = True
    )
    trainer = Trainer(
        device=device,tasks=[target_task],
        data_args=data_args,model_path=model_path)
    _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
    result = {'task': target_task, 'source1': source_task1, 'source2': source_task2}
    for metric in data_args['metrics']:
        if metric in ['r2', 'mae']:
          result[metric] = task_results_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

h298 u0
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
initializing trainer...
[0] training loss:1.6178422713279723
h298
val r2:0.34100165022289886
val mae:25.024396896362305
u0
val r2:0.3354895607153259
val mae:25.114574432373047
[20] training loss:0.39678486973047256
h298
val r2:0.794492363585289
val mae:16.01349639892578
u0
val r2:0.7994370568900999
val mae:14.502849578857422
[40] training loss:0.31062722355127337
h298
val r2:0.8422441171071462
val mae:14.020047187805176
u0
val r2:0.8361296044632921
val mae:14.20901870727539
[60] training loss:0.2835249376296997
h298
val r2:0.8554266492731626
val mae:13.296500205993652
u0
val r2:0.8568521416216527
val mae:13.119257926940918
[80] training loss:0.24483759760856627
h298
val r2:0.8795511132998339
val mae:12.001932144165039
u0
val r2:0.8750815124275337
val mae:12.141040802001953
[100] training loss:0.19987702935934068
h298
val r2:0.9113379650611437
val mae:10.120559692382812
u0
val r2:0.9113042573327934
val mae

In [ ]:
#Pre-train GNN w/ target task and second source task, using pre-trained weights from target task + first source task

model_type = 'GCN'
model_path = second_train_model_path
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[], 'helper_task':[], 'helper_task2':[]}

for source_task1, source_task2, _, target_task in source_target_pairs:
    print('multi ' + target_task + ' ' + source_task2)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[target_task],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task2],
        device = device
    )
    model = load_model(device=device, is_multi = True, source_model_path=first_train_model_path+f"{target_task}_{source_task1}.pth")
    trainer = Multi_Trainer(device=device,task1=target_task,task2=source_task2,
                      data_args1=data_args1,data_args2=data_args2,model_path=model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)

#Use target data to fine-tune model pre-trained on 2 source tasks + target

results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'r2', 'mae'])
for source_task1, source_task2, _, target_task in source_target_pairs:
    print(f"{target_task},{source_task1}->{target_task}")

    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,data_path=data_path,tasks=[target_task],
        device = device
    )
    model = load_model(
        device=device,
        source_model_path=second_train_model_path+f"{target_task}_{source_task2}.pth",
        is_multi = True
    )
    trainer = Trainer(
        device=device,tasks=[target_task],
        data_args=data_args,model_path=model_path)
    _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
    result = {'task': target_task, 'source1': source_task1, 'source2': source_task2}
    for metric in data_args['metrics']:
        if metric in ['r2', 'mae']:
          result[metric] = task_results_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

h298 g298
800 loaded!
100 loaded!
1000 loaded!
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000/h298_u0.pth
initializing trainer...
[0] training loss:0.04301163433119655
h298
val r2:0.9883552788805944
val mae:3.7823305130004883
g298
val r2:0.9884233535365057
val mae:3.884126663208008
[20] training loss:0.025931605771183967
h298
val r2:0.9884994089123662
val mae:3.856518507003784
g298
val r2:0.9891426856193657
val mae:3.7116494178771973
[40] training loss:0.023373994547873735
h298
val r2:0.9875106895409741
val mae:4.0510969161987305
g298
val r2:0.9848501752436996
val mae:4.535268783569336
test r2:0.9763305465438605
test mae:3.817281484603882
test r2:0.9731275072027897
test mae:3.9217076301574707


In [ ]:
#Pre-train GNN w/ target task and third source task, using pre-trained weights from target task + first two source tasks

model_type = 'GCN'
model_path = third_train_model_path
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[], 'helper_task':[], 'helper_task2':[]}

for source_task1, source_task2, source_task3, target_task in source_target_pairs:
    print(target_task + ' ' + source_task3)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[target_task],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task3],
        device = device
    )
    model = load_model(device=device, is_multi = True, source_model_path=first_train_model_path+f"{target_task}_{source_task1}.pth")
    trainer = Multi_Trainer(device=device,task1=target_task,task2=source_task3,
                      data_args1=data_args1,data_args2=data_args2,model_path=model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)

#Use target data to fine-tune model pre-trained on 2 source tasks + target

results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'source3', 'r2', 'mae'])
for source_task1, source_task2, source_task3, target_task in source_target_pairs:
    print(f"{target_task},{source_task1},{}->{target_task}")

    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,data_path=data_path,tasks=[target_task],
        device = device
    )
    model = load_model(
        device=device,
        source_model_path=second_train_model_path+f"{target_task}_{source_task3}.pth",
        is_multi = True
    )
    trainer = Trainer(
        device=device,tasks=[target_task],
        data_args=data_args,model_path=model_path)
    _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
    result = {'task': target_task, 'source1': source_task1, 'source2': source_task2, 'source3': source_task3}
    for metric in data_args['metrics']:
        if metric in ['r2', 'mae']:
          result[metric] = task_results_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

h298,u0->h298
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi1000->multi1000/h298_g298.pth
[0] training loss:0.014255622196942567
val r2:0.9840718141307033
val mae:4.894855976104736
[20] training loss:0.013047841656953097
val r2:0.9881733275489134
val mae:3.8517274856567383
test r2:0.9772722997186876
test mae:3.665956497192383
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/QM9/GCN/multi1000->multi1000->1000/results.csv


<ipython-input-33-7c524f410d01>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [ ]:
#OLD: Pre-train GNN on selected source tasks

model_type = 'GCN'
model_path = source_model_path
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[], 'helper_task':[]}

for source_task1, source_task2, _ in source_target_pairs:
    print(source_task1 + ' ' + source_task2)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task1],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task2],
        device = device
    )
    model = load_model(device=device, is_multi = True)
    trainer = Multi_Trainer(device=device,task1=source_task1,task2=source_task2,
                      data_args1=data_args1,data_args2=data_args2,model_path=model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)
    results_dict['task'].append(source_task1)
    results_dict['task'].append(source_task2)
    results_dict['helper_task'].append(source_task2)
    results_dict['helper_task'].append(source_task1)
    for metric in data_args1['metrics']: #Share the same metric in this case
        if metric not in list(results_dict.keys()):
            results_dict.update({metric:[]})
        results_dict[metric].append(task_results1_dict[metric][source_task1])
        results_dict[metric].append(task_results2_dict[metric][source_task2])
    result_path = model_path.replace('saved_models','results')
    makedir(result_path)
    pd.DataFrame(results_dict).to_csv(result_path+'results.csv', float_format='%.3f',
                                      index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

zpve r2
8000 loaded!
1000 loaded!
1000 loaded!
8000 loaded!
1000 loaded!
1000 loaded!
initializing trainer...
[0] training loss:1.6141012041568756
zpve
val r2:0.44371404068346
val mae:0.02046278491616249
r2
val r2:0.06962152844087566
val mae:194.35964965820312
[20] training loss:1.0693679540157317
zpve
val r2:0.5964868051595731
val mae:0.0171906016767025
r2
val r2:0.2979038500855027
val mae:158.6237030029297
[40] training loss:0.9930933074951172
zpve
val r2:0.610806198218965
val mae:0.016995703801512718
r2
val r2:0.35672208391285976
val mae:167.91378784179688
[60] training loss:0.945448703289032
zpve
val r2:0.61662825143478
val mae:0.01685425452888012
r2
val r2:0.42473405456846824
val mae:148.98519897460938
[80] training loss:0.9105422267913819
zpve
val r2:0.6246746950070912
val mae:0.0165560282766819
r2
val r2:0.4423073224789734
val mae:144.17636108398438
[100] training loss:0.8602569547891616
zpve
val r2:0.6205007112132154
val mae:0.01670532114803791
r2
val r2:0.4742685562665496
val 

NameError: ignored

In [ ]:
#Train pre-trained GNN on target task

results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'r2', 'mae'])
for source_task1, source_task2, target_task in source_target_pairs:
    print(f"{source_task1},{source_task2}->{target_task}")

    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,data_path=data_path,tasks=[target_task],
        device = device
    )
    model = load_model(
        device=device,
        source_model_path=source_model_path+f"{source_task1}_{source_task2}.pth",
        is_multi = True
    )
    trainer = Trainer(
        device=device,tasks=[target_task],
        data_args=data_args,model_path=model_path)
    _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
    result = {'task': target_task, 'source1': source_task1, 'source2': source_task2}
    for metric in data_args['metrics']:
        if metric in ['r2', 'mae']:
          result[metric] = task_results_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
result_path = model_path.replace('saved_models','results')
makedir(result_path)
results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

zpve,r2->alpha
800 loaded!
100 loaded!
1000 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/QM9/GCN/multi10000/zpve_r2.pth
[0] training loss:0.3315845802426338
val r2:0.6291150512565564
val mae:3.986130475997925
[20] training loss:0.19351140171289444
val r2:0.7836728065570908
val mae:2.9752025604248047
[40] training loss:0.18181807704269887
val r2:0.793402291664573
val mae:2.8578860759735107
[60] training loss:0.1797895273566246
val r2:0.7749505826967089
val mae:3.0141282081604004
[80] training loss:0.16177251935005188
val r2:0.7890597922831676
val mae:2.969271183013916
[100] training loss:0.15814108297228813
val r2:0.8001688042081471
val mae:2.8492157459259033
[120] training loss:0.145843545794487
val r2:0.7543424241709338
val mae:3.184645891189575
test r2:0.7568098526567733
test mae:2.9021036624908447
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/QM9/GCN/multi10000->1000/resu

<ipython-input-63-af2316158fbd>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)
